In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import open3d as o3d

def get_grid_points(chgcar):
    grid = chgcar.data['total']
    dims = grid.shape
    return [np.array([i, j, k]) for i in np.linspace(0, 1, dims[0]) for j in np.linspace(0, 1, dims[1]) for k in np.linspace(0, 1, dims[2])]

def grid_points_to_cartesian(lattice, grid_points):
    return [lattice.get_cartesian_coords(gp) for gp in grid_points]


In [ ]:
import json
with open("mp_id_data.json", "r") as f:
    dic = json.load(f)

In [ ]:
import pickle
with open(file=tot[0], mode='rb') as f:
    chrgecar = pickle.load(f)

In [ ]:
key = list(chrgecar.spin_data.keys())

In [ ]:
data = np.concatenate((cartesian_points, np.expand_dims(total_data, axis=1),  ),axis=1)

In [ ]:
#1feat
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
import random
from concurrent.futures import ProcessPoolExecutor
import glob
import os
import pickle

tot = glob.glob("/path/to/saved/*.pickle")

def make_np(i, dic):
    #print(i)
    with open(file=i, mode='rb') as f:
        chrgecar = pickle.load(f)
    chgcar = chrgecar
    grid_points = get_grid_points(chgcar)
    cartesian_points = grid_points_to_cartesian(chgcar.structure.lattice, grid_points)

    total_data = chgcar.data['total'].flatten()/dic[i.split('/')[-1].split('.')[0]]['vol']
    data = np.concatenate((cartesian_points, np.expand_dims(chgcar.data['total'].flatten(), axis=1)),axis=1)
    data =  data.astype(np.float16)
    file = "/path/to/save/%s.npy" %(i.split('/')[-1].split('.')[0])
    np.save(file, data)


In [ ]:
#4feat
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
import random
from concurrent.futures import ProcessPoolExecutor
import glob
import os
import pickle

tot = glob.glob("/path/to/saved/*.pickle")

def make_np(i, dic):
    #print(i)
    with open(file=i, mode='rb') as f:
        chrgecar = pickle.load(f)
    chgcar = chrgecar
    grid_points = get_grid_points(chgcar)
    cartesian_points = grid_points_to_cartesian(chgcar.structure.lattice, grid_points)
    key = list(chrgecar.spin_data.keys())
    total_data = chgcar.data['total'].flatten()/dic[i.split('/')[-1].split('.')[0]]['vol']
    total_data1 = chgcar.data['diff'].flatten()/dic[i.split('/')[-1].split('.')[0]]['vol']
    total_data2 = chgcar.spin_data[key[0]].flatten()/dic[i.split('/')[-1].split('.')[0]]['vol']
    total_data3 = chgcar.spin_data[key[1]].flatten()/dic[i.split('/')[-1].split('.')[0]]['vol']
    data = np.concatenate((cartesian_points, np.expand_dims(total_data, axis=1), 
                           np.expand_dims(total_data1, axis=1), np.expand_dims(total_data2, axis=1),
                           np.expand_dims(total_data3, axis=1)), axis=1)
    data =  data.astype(np.float16)
    #file = "/path/to/save/%s.npy" %(i.split('/')[-1].split('.')[0])
    #np.save(file, data)
    return data

In [ ]:
for i in tot:
    make_np(i, dic)

In [ ]:
from torch_geometric.data import Batch, Data
from tqdm.auto import tqdm
import torch
import torch_geometric.transforms as T
import torch_points3d.core.data_transform as T3D
import pickle
import numpy as np
import glob
#pre_transform = T.Compose([T.NormalizeScale()])
pt_data = []
pre_transform1 = T.Compose([T3D.GridSampling3D(0.25,  quantize_coords=True, verbose=True)])
tot = glob.glob("/path/to/saved/*.npy")
for idx_batch in tqdm(range(len(tot))):
        data = np.load(tot[idx_batch])
        point_cloud = torch.from_numpy(data).to(dtype=torch.float32)
        point_cloud = point_cloud[point_cloud[:, 3] > 0]
        sampled_point_cloud = point_cloud
        pos = sampled_point_cloud[:, :3]
        t = sampled_point_cloud[:, 3:]/dic[tot[idx_batch].split('/')[-1].split('.')[0]]
        tar = torch.tensor([target], dtype=torch.int32)
        data = pre_transform1(Data(pos=pos, x=t))
        torch.save(data, '/path/to/save/mp_cart_rand_pt/' + i.split('/')[-1].split('.')[0] + '.pt')